In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import json

In [3]:
file = open("./dataset/glassdoor_reviews.json", 'r', encoding='utf8')
dataset = json.load(file)

In [5]:
print("Dataset Size: %d" % (len(dataset)))
print(f"Sample Item: {dataset[0]}")

Dataset Size: 838566
Sample Item: {'firm': 'AFH-Wealth-Management', 'date_review': '2015-04-05', 'job_title': ' ', 'current': 'Current Employee', 'location': '', 'overall_rating': 2, 'work_life_balance': 4.0, 'culture_values': 3.0, 'diversity_inclusion': '', 'career_opp': 2.0, 'comp_benefits': 3.0, 'senior_mgmt': 3.0, 'recommend': 'x', 'ceo_approv': 'o', 'outlook': 'r', 'headline': 'Young colleagues, poor micro management', 'pros': 'Very friendly and welcoming to new staff. Easy going ethic.', 'cons': 'Poor salaries, poor training and communication.'}
